In [2]:
import numpy as np
import scipy as sp
import cmath

**Part 1: Bitwise operations** $$ \\ $$
Let us see how to represent a spin $\frac{1}{2}$ chain (or a chain of spinless fermions) using integers and their bit representation. First, let us see how to get the bitstring out of an integer:

In [3]:
print(bin(8))
#or
print(format(8,'b'))

0b1000
1000


Using bitwise operators, create a function that shifts every state to the right by $n$ units:

In [4]:
def shift_right(i:int,n:int,N:int):
    for j in range(0,n):
        i=(i>>1)+2**(N-1)*(i%2)
    return(i)


Let us time it for later comparison

In [5]:
%timeit shift_right(4,2,4)

796 ns ± 29.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Let us write the same function for an array, using np.roll:

In [6]:
def shift_right_array(a:np.array,n):
    return(np.roll(a,n))

Let us time it as well and compare:

In [7]:
ar=np.array([1,0,1,0])
%timeit shift_right_array(ar,2)

6.22 µs ± 80.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Great, we can actually save quite some time by resorting to the representation of states as bits of an integer. Similarly to above, create a function that shifts every state to the left by $n$ units:

In [8]:
def shift_left(i:int,n:int,N:int):

    for j in range(0,n):
        if i>=2**(N-1):

            i=(i<<1) -(2**(N)) +1
        else:
            i=i<<1
    return(i)

Write a function to count the number of up spins:


In [9]:
def count_upspins(i:int):
    return(format(i,'b').count('1'))

Write a function to check the spin at site j:

In [10]:
def spin_at_j(i:int,j:int):
    return((i>>j)%2)

Write a function to flip the spin at site j:

In [11]:
def flip_spin(i:int,j:int):
    return((1<<j)^i)

Write a function to flip all the spins at once:

In [12]:
def flip_all_spins(i:int,N:int):
    return(i^(2**N-1))

Assuming we have a chain of 10 spin-$\frac{1}{2}$'s, compute the sizes of the different magnetization sectors of the Hilbert space.

In [13]:
#state the size of your system
N=10

#state the possible values the magnetization may take
possible_magnetizations=np.arange(-N,N+1,2)

#Create a dictionary to save the set of states for each magnetization sector
m_sectors={}

#Initialize an empty list in the dictionary for every possible m-value
for m in possible_magnetizations:
    m_sectors[f'{m}']=[]


#Iterate over all the basis states and order them according to their magnetization
for n in range(0,2**N):
    n_up=count_upspins(n)
    n_down=N-n_up
    m=n_up-n_down
    m_sectors[f'{m}'].append(n)

#print the dimensions of the subspaces for the different magnetization sectors

for m,basis in m_sectors.items():
    print(m,len(basis))

-10 1
-8 10
-6 45
-4 120
-2 210
0 252
2 210
4 120
6 45
8 10
10 1


**Part 2: Transverse Field Ising Model**

By now you should have solved the analytical exercise to express the Transverse Field Ising Hamiltonian in the momentum state basis. Let us apply what we have learned to the numerical study of the TFIM,
$$\hat H = \sum\limits_{i=0}^{N - 1} \sigma_i^z \sigma_{i + 1}^z + h^x \sum\limits_i \sigma^x.$$
The ultimate goal will be to exactly diagonalize the model and make use of the translation symmetry. Here we'll do some preparatory work. Assume $N=8$. What are the possible momenta $P_n$ that the system can have?$ \\ $
$ P_n=\frac{2\pi}{N}n, \; n=0,1,...,7.$ For every momentum $P_n$: $ \\ $-Find the representatives of all the orbits generated by the translation operator $\hat{T}$. Choose as representative of an orbit the state of the orbit corresponding to the lowest integer value (state with up- and down spins <--> integer value via the bit representation). $ \\ $ -Next, check which orbits are compatible with the momentum $P_n$ (remember that e.g. the orbit created by the state $|\uparrow\downarrow\uparrow\downarrow\uparrow\downarrow\uparrow\downarrow\rangle$ is only compatible with the momenta $P_0=\frac{2\pi}{8}0=0$ and $P_2=\frac{2\pi}{8}4=\pi$, because translating the state twice must bring it back to itself, meaning that $\exp(-iP_n2)$ must be equal to $1$.)


In [ ]:
#Write a function that gives you the representatives for every orbit:
def orbit_representatives(N:int):

    reps=[]

    for i in range(0,2**N):

        rep_i=2**N

        for j in range(0,N):

            moved_state=shift_right(i,j,N)

            if moved_state < rep_i:

                rep_i=moved_state

        if rep_i not in reps:
            reps.append(rep_i)

    return(np.array(reps))


#Using the array with representatives, i.e. the output of orbit_representatives, check which orbits
#are compatible with the momentum P_n and return the representatives:

def compatible_representatives(reps:np.array,n:int,N:int):

    compat_reps=[]

    for rep in reps:
         #check how many different states are in an orbit:
            counter=0
            state=rep
            new_state=2**N

            while new_state!=rep:

                new_state=shift_right(state,1,N)
                state=new_state
                counter+=1
                #print(new_state)

            if (counter*n/N).is_integer():

                compat_reps.append(rep)
                print("rep possible", compat_reps)

    return(np.array(compat_reps))




In [19]:
print(orbit_representatives(3))
print(compatible_representatives(orbit_representatives(3),2,3))

[0 1 3 7]
0
4
2
1
rep possible [np.int64(1)]
5
6
3
rep possible [np.int64(1), np.int64(3)]
7
[1 3]


The number of representatives that are compatible with a certain momentum is also the dimension of the corresponding invariant subspace (make sure that you understand that it is not a basis for the invariant subspace, though...). Check that everything adds up (in the case N=8), i.e. that the compatible states for all the momenta sum up to 256.

In [ ]:
#Define the system size:
N=8

#initialize a counter
counter=0

for n in range(0,N):
    counter+=len(compatible_representatives(orbit_representatives(N),n,N))

print(counter)

